In [ ]:
import sagemaker
import lightgbm as lgb
import pandas as pd
import boto3

In [ ]:
session = boto3.Session()

In [ ]:
sm = sagemaker.Session(boto_session=session)

In [ ]:
sm._region_name

In [ ]:
train_df = pd.read_csv('~/Downloads/train.csv')
test_df = pd.read_csv('~/Downloads/test.csv')

In [ ]:
import pandas_profiling

from pandas_profiling import ProfileReport

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from lightgbm.sklearn import LGBMClassifier

In [ ]:
pr = ProfileReport(train_df, )
pr

In [ ]:
num_columns = ['Age', 'Fare']

In [ ]:
median_imputer = SimpleImputer(strategy='median')

In [ ]:
cat_columns = ['Sex', 'Embarked', 'Pclass']

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore')

In [ ]:
ct = ColumnTransformer(
    transformers=[
        (
            'num_columns', median_imputer, num_columns,
        ),
        (
            'cat_columns', ohe, cat_columns
        )
    ]

)

In [ ]:
pipeline = Pipeline(
    steps=[
        ('preprocess', ct),
        ('model', LGBMClassifier(n_estimators=150))
    ]
)

In [ ]:
train_df = train_df[cat_columns + num_columns + ['Survived']]
test_df = test_df[cat_columns + num_columns]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_df.drop(columns=['Survived']),
                                                    train_df.Survived, shuffle=True)

In [ ]:
cv = cross_val_score(pipeline, x_train, y_train, scoring='accuracy', cv=5)
cv.mean(), cv.std()